## BETTER WAY 29 - 게터와 세터 메소드 대신에 일반 속성을 사용하자
## Item29 - Use Plain Attributes Instead of Get and Set Methods

자바나 다른 언어에서 파이썬으로 넘어온 프로그래머들은 자연스레 클래스에 게터(getter)와 세터(setter) 메서드를 명시적으로 구현하려고 한다. (그런가?...)

In [1]:
class OldResistor(object):
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms

이런 게터와 세터를 사용하는 방법은 간단하지만 파이썬답지 않다.

In [2]:
r0 = OldResistor(50e3)
print('Before: %5r' % r0.get_ohms())
r0.set_ohms(10e3)
print('After: %5r' % r0.get_ohms())

Before: 50000.0
After: 10000.0


게터와 세터 메서드는 특히 즉석에서 증가시키기 같은 연산에는 사용하기 불편하다.

In [3]:
r0.set_ohms(r0.get_ohms() + 5e3)
print('After: %5r' % r0.get_ohms())

After: 15000.0


 파이썬에서는 명시적인 게터와 세터를 구현할 일이 거의 없다. 대신 항상 간단한 공개 속성부터 구현하기 시작해야 한다.

In [4]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0
        

r1 = Resistor(50e3)
r1.ohms = 10e3

이렇게 하면 즉석에서 증가시키기 같은 연산이 자연스럽고 명확해진다.

In [5]:
r1.ohms += 5e3
print(r1.ohms)

15000.0


파이썬에서는 `@property` 데코레이터(decorator)를 이용해 getter와 setter 속성을 사용하는 방법이 있다. 단, 제대로 동작하려면 세터와 게터 메서드의 이름이 프로퍼티 이름과 일치해야 한다.

In [6]:
class VoltageReistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
        
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

In [7]:
r2 = VoltageReistance(1e3)
print('Before: %5r amps' % r2.current)
r2.voltage = 10
print('After: %5r amps' % r2.current)

Before:     0 amps
After:  0.01 amps


`@property`에 `setter`를 설정하면 클래스에 전달된 값들의 타입을 체크하고 값을 검증할 수 있다. 아래의 클래스는 모든 저항값이 0옴보다 크다는 것을 나타내는 클래스이다.

In [8]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
            self._ohms = ohms

속성에 올바르지 않은 저항값을 할당하면 예외가 일어난다.

In [9]:
r3 = BoundedResistance(1e3)
r3.ohms = 0

ValueError: 0.000000 ohms must be > 0

생성자에 올바르지 않은 값을 넘겨도 예외가 일어난다.

In [10]:
BoundedResistance(-5)

ValueError: -5.000000 ohms must be > 0

부모클래스의 속성을 불변(immutable)으로 만드는 데에도 `@property`를 사용할 수 있다.

In [11]:
class FixedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, '_ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms

이 객체를 생성하고 나서 프로퍼티에 할당하려고 하면 예외가 일어난다.

In [12]:
r4 = FixedResistance(1e3)
r4.ohms = 2e3

AttributeError: Can't set attribute

`@property`의 가장 큰 단점은 속성에 대응하는 메서드를 서브클래스에서만 공유할 수 있다는 점이다. 서로 관련이 없는 클래스는 같은 구현을 공유하지 못한다.

### 정리
- 간단한 공개 속성을 사용하여 새 클래스 인터페이스를 정의하고 세터와 게터 메서드는 사용하지 말자.
- 객체의 속성에 접근할 때 특별한 동작을 정의하려면 `@property`를 사용하자.